# Tencent VectorDB Python SDK

## 打包并本地安装

In [ ]:
%pip install wheel
!python3 setup.py sdist bdist_wheel || true
!pip3 install dist/tcvectordb-0.0.2-py3-none-any.whl

## 连接VectorDB

In [ ]:
import tcvectordb
cli = tcvectordb.VectorDB(url='', username='root',
                        key='', timeout=5)

## Manage Databases

In [ ]:
# Create database
cli.create_database('db001')
# List databases
cli.list_databases()

In [ ]:
# Drop database
cli.drop_database('db001')
# List databases
cli.list_databases()

## Manage Collection

In [ ]:
from tcvectordb.model.collection import IndexType
from tcvectordb.model.collection import MetricType
from tcvectordb.model.enum import FieldType
from tcvectordb.model.index import Index, VectorIndex, ScalarIndex, HNSWParams


index = Index().add(ScalarIndex('id', FieldType.String, IndexType.PRIMARY_KEY))
index.add(VectorIndex('vector', 3, IndexType.HNSW,
            MetricType.L2, HNSWParams(64, 8)))
index.add(ScalarIndex('other', FieldType.String, IndexType.FILTER))

db = cli.create_database('db001')
db.create_collection(
    name='col0',
    shard=3,
    replicas=2,
    description='test collection0',
    index=index,
    timeout=20
)
db.create_collection(
    name='col1',
    shard=3,
    replicas=2,
    description='test collection1',
    index=index,
    timeout=20
)
col_list = db.list_collections()
for elem in col_list:
    print(vars(elem))

col = db.describe_collection('col1')
vars(col)
print(col.database_name, col.collection_name, col.shard, col.replicas, col.description, col.create_time)
print(col.index)

db.drop_collection('col1')
db.describe_collection('col1')

## Manage Document

In [ ]:
from tcvectordb.model.document import Filter

def filter():
    filter = Filter('age > 18').And(
        'gender = "male"').Or(
        'city = "Beijing"').AndNot(
        'age > 30').OrNot('city = "Shanghai"').And(Filter.In('id', ['123','456']))
    
    print(filter.cond)

filter()

In [ ]:
from tcvectordb.model.document import Document

col = db.database('db001').collection('col0')
col.upsert(
    documents=[
        Document(id='0001', vector=[0.2123, 0.23, 0.213], other='aa'),
        Document(id='0002', vector=[0.5123, 0.53, 0.513], other='bb'),
        Document(id='0003', vector=[0.9123, 0.93, 0.913], other='aa'),
    ],
    build_index=False,
)

In [ ]:
from tcvectordb.model.document import Document
col = db.database('db001').collection('col0')

res = col.query(primary=['0001', '0002'], retrieve_vector=True)
for elem in res:
    print(vars(elem))

In [ ]:
from tcvectordb.model.document import Filter
col = db.database('db001').collection('col0')
filter = Filter('other="aa"')
vecs = col.search(vector={'vectors': [[0.3123, 0.43, 0.213]]},
    filter=filter, retrieve_vector=False, limit=2)
for arr in vecs:
    print(vars(arr))

col.delete(['0001'])

## 环境清除

In [ ]:
db.drop_database('db001')
!pip3 uninstall -y tcvectordb